# Créer un Vector Store
---

In [ ]:
# Installer Faiss (version CPU)
!pip install faiss-cpu --quiet

# Installer le client Mistral
!pip install mistralai --quiet

In [ ]:
import os
os.environ["MISTRAL_API_KEY"] = ""

In [ ]:
from mistralai import Mistral
import os
import numpy as np

# Initialisation du client avec votre clé API
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

# Fonction pour générer l'embedding d'un texte
def embed_text(text):
    # Utilisation de la méthode 'create' de l'objet 'embeddings'
    response = client.embeddings.create(
        model="mistral-embed",
        inputs=[text]
    )
    return response.data[0].embedding

# Exemple de documents à indexer
documents = [
    "La mairie est ouverte du lundi au vendredi de 9h à 17h.",
    "Le règlement municipal interdit le stationnement devant les écoles.",
    "Les horaires des déchetteries varient selon les saisons.",
    "Les demandes de passeport se font uniquement sur rendez-vous.",
    "Les aides sociales sont disponibles sous conditions de revenus."
]

# Génération des embeddings pour chaque document
embeddings = np.array([embed_text(doc) for doc in documents])

In [ ]:
import faiss

# Dimension des vecteurs (embedding size)
dimension = embeddings.shape[1]

# Création d'un index Faiss L2 (distance euclidienne)
index = faiss.IndexFlatL2(dimension)

# Ajout des vecteurs à l'index
index.add(embeddings)

# Sauvegarde éventuelle
faiss.write_index(index, "faiss_index.idx")

In [ ]:
# Requête utilisateur
query = "Quels sont les horaires d'ouverture de la mairie ?"

# Génération de l'embedding de la requête
query_embedding = embed_text(query)

# Recherche des 3 documents les plus proches
k = 3
distances, indices = index.search(np.array([query_embedding]), k)

# Affichage des résultats
print("Résultats les plus pertinents :\n")
for i, idx in enumerate(indices[0]):
    print(f"📝 Document {idx} (distance: {distances[0][i]:.4f}) :\n{documents[idx]}\n")